## Structuring classes which follow a simple protocol

In [47]:
from dataclasses import dataclass, field
from cattr.preconf.pyyaml import make_converter
from typing import Protocol, ClassVar

class P(Protocol):
    class_id: ClassVar[str]

_class_dict = {}
def register_for_converter(cls):
    _class_dict[cls.class_id] = cls
    return cls

@register_for_converter
@dataclass
class A:
    class_id: ClassVar[str] = "A"
    a: str
    b: int = 1
    
@register_for_converter
@dataclass
class B:
    class_id: ClassVar[str] = "B"
    c: str = "hallo"

@dataclass
class XList:
    l: list[P]

converter = make_converter(omit_if_default=True)
converter2 = make_converter(omit_if_default=True)
def unstructure_P(p, *args):
    dd = converter2.unstructure(p, *args)
    dd.update({"type": p.class_id})
    return dd
def structure_P(data: dict, *args):
    cls = _class_dict[data["type"]]
    return converter2.structure(data, cls)

converter.register_unstructure_hook(P, unstructure_P)
converter.register_structure_hook(P, structure_P)

### Lets try it

In [51]:
a = A(a="hallo", b=2)
b = B(c="hallo welt")
l = XList([a,b])
a_s = converter.unstructure(a, P)
b_s = converter.unstructure(b, P)
l_s = converter.unstructure(l)
print(a_s)
a1 = converter.structure(a_s, P)
l2 = converter.structure(l_s, XList)
print(a, a1)
print(l, l2)

{'a': 'hallo', 'b': 2, 'type': 'A'}
A(a='hallo', b=2) A(a='hallo', b=2)
XList(l=[A(a='hallo', b=2), B(c='hallo welt')]) XList(l=[A(a='hallo', b=2), B(c='hallo welt')])


In [42]:
help(converter.copy)

Help on method copy in module cattrs.converters:

copy(dict_factory: Optional[Callable[[], Any]] = None, unstruct_strat: Optional[cattrs.converters.UnstructureStrategy] = None, omit_if_default: Optional[bool] = None, forbid_extra_keys: Optional[bool] = None, type_overrides: Optional[collections.abc.Mapping[Type, cattrs.gen.AttributeOverride]] = None, unstruct_collection_overrides: Optional[collections.abc.Mapping[Type, Callable]] = None, prefer_attrib_converters: Optional[bool] = None, detailed_validation: Optional[bool] = None) -> 'Converter' method of cattrs.preconf.pyyaml.PyyamlConverter instance

